In [1]:
import torch

<img src="../images/mlp.png" alt="drawing" width="500"/><br>

# Forward pass

In [122]:
# Input data. 2 example with 3 features.
x = torch.tensor([[1, 0, 1],
                  [2, 1, 0],
                  [1, 0, 1],
                  [2, 1, 0]], dtype=torch.float)
y = torch.tensor([[1, 0],
                  [0, 1],
                  [1, 0],
                  [0, 1]], dtype=torch.float)

# weitght matrix of first layer. dim (3, 2)
w1 = torch.tensor([[0.5, 0.2],
                   [0.2, 0.3],
                   [0.1, 0.1]])
# First layer bias dim (2, )
b1 = torch.tensor([[0.5, 0.7]])
# Second layer weight matrix dim (2, 3)
w2 = torch.tensor([[0.1, 0.2, 0.5],
                   [0.1, 0.3, 0.1]])
# Second layer bias dim (3, )
b2 = torch.tensor([[0.3,0.6,0.7]])
# weitght matrix of third layer. dim (3, 2)
w3 = torch.tensor([[0.5, 0.2],
                   [0.2, 0.3],
                   [0.1, 0.1]])
# Third layer bias dim (2, )
b3 = torch.tensor([[0.5,0.7]])

In [123]:
# forward pass layer 1
preact1_a1 = x @ w1 + b1
postact1_h1 = torch.sigmoid(preact1_a1)
# forward pass layer 2
preact2_a2 = postact1_h1 @ w2 + b2
postact2_h2 = torch.sigmoid(preact2_a2)
# forward pass layer 3
preact3_yhat = postact2_h2 @ w3  + b3
yhat = torch.softmax(preact3_yhat, dim=1)

# Back Propagation

**Gradient Descent**<br>
Assume the problem is a k clas classifictaion problem. There are total N training sample, each sample has d features.<br>
The complete calculation is there in the deep learning notevook<br>
1. Loss Function: Cross Entropy loss ($L\in R$)
$$
\begin{align*}
L&=-\sum_{i=1}^N \sum_{c=1}^k y_{i,c} log(\hat{y}_{i,c})\\\
&=-\sum_{i=1}^N log(\hat{y}_{i,l}) \text{where, l corrcsponds to the true class for the example i}
\end{align*}
$$

2. Gradient corresponding to predicted value (output vector for $i^{th} example$) ($\nabla_{\hat{y_i}}L\in R^k$)
$$
\nabla_{\hat{y_i}}L = -\frac{1}{\hat{y}_{i,l}}e_{i,l}   \text{    where  }e_{i,l} \in R^k=\begin{cases}1 \text{  if  } l=i\\0 \text{ else } \end{cases}\\
$$
Therefore, $\nabla_{\hat{y}}\in R^{N\times k}$, stack gradients corresponding to each example<br>
3. Gradient corresponding to the preactivation part of final layer and $i^{th} example$($\nabla_{a_i^L}\in R^k$)
$$
\nabla_{a_i^L} = \hat{y}_i-e_{i, l}
$$
Therefore, $\nabla_{a^L}\in R^{N\times k}$, stack gradients corresponding to each example<br>
4. Gradient corresponding to activations of $i^{th}$ example $2^{nd}$ layer ($\nabla_{h_i^2}\in R^3$)
In general $i^{th}$ example $j^{nd}$
$$
\nabla_{h_i^j} = \nabla_{a^{j+1}}LW^{j+1^{T}}\\
\nabla_{h_i^2} = \nabla_{a^{3}}LW^{3^T}
$$
Therefore, $\nabla_{h^2}\in R^{N\times 3}$, stack gradients corresponding to each example<br>
5. Gradient corresponding to pre activation of $i^{th}$ example $2^{nd}$ layer ($\nabla_{a_i^2}\in R^3$)
In general $i^{th}$ example $j^{nd}$
$$
\nabla_{a_i^j} = \nabla_{h^{i}}L\odot g^{'}(a^i)\\
\nabla_{a^2_i} = \nabla_{h^{2}_i}L\odot g^{'}(a^2_i)\\
$$
Therefore, $\nabla_{a^2}\in R^{N\times 3}$, stack gradients corresponding to each example<br>
6. Gradient correspondint to weights ($\nabla_{W^3}\in R^{3\times 2}$)
$$
\nabla_{W^i}L=h^{(j-1)^T}\nabla_{a^i}L\\
\nabla_{W^3}L=h^{(2)^T}\nabla_{a^3}L
$$
7. Gradient correspondint to biases ($\nabla_{b^3}\in R^{2}$)
$$
\nabla_{b^i}L=\frac{1}{N}\sum_{j=1}^{N}\nabla_{a^i_j}L\\
\nabla_{b^3}L=\frac{1}{N}\sum_{j=1}^{N}\nabla_{a^3_j}L\\
$$

In [124]:
# Backpropagation
loss = -torch.nn.functional.cross_entropy(y, torch.log(yhat))
print(loss)

tensor(1.1289)


In [148]:
# gradient corresponding to the parameters of the final layer
class_labels = torch.argmax(y, dim=1)   # Find the labels for each example.
grad_yhat = torch.zeros_like(yhat)
grad_yhat[torch.arange(len(class_labels)), class_labels] = -1/yhat[torch.arange(len(class_labels)), class_labels]
# grad_yhat = torch.tensor([[0.48, 0],
#                          [0, 0.52]])
grad_a3 = yhat - y
grad_w3 = postact2_h2.T @ grad_a3
grad_b3 = grad_a3.mean(dim=0)

In [153]:
postact2_h2.shape

torch.Size([4, 3])

In [145]:
# gradient corresponding to the parameters of the final layer
grad_h2 = grad_a3 @ w3.T
grad_a2 = grad_h2 * preact2_a2
grad_w2 = postact1_h1.T @ grad_a2
grad_b2 = grad_a2.mean(dim=0)

In [147]:
grad_b2

tensor([-1.8051e-03,  1.1965e-03, -6.4163e-09])